In [25]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_DIR = './../data' 
STUDY_DIR = DATA_DIR + '/study'


users = pd.read_csv(f'{DATA_DIR}/users.csv.gz')

# use study for less data, for testing
events = pd.read_csv(f'{STUDY_DIR}/events.csv.gz')
transactions = pd.read_csv(f'{STUDY_DIR}/transactions.csv.gz')

In [27]:
from data import preprocess_transactions, get_transactions_dataloader

train_transactions, test_transactions, user_ids, topic_ids = preprocess_transactions(transactions)

train_dataloader = get_transactions_dataloader(train_transactions, 256)

test_dataloader = get_transactions_dataloader(test_transactions, 256)


In [28]:
import os
import torch
from torch import optim, nn
import lightning.pytorch as pl
from ncf_model import NCFNetwork


# TODO: fill these
num_students = len(user_ids)
num_topics = len(topic_ids)

student_embedding_dim = 32
topic_embedding_dim = 32


# init
ncf = NCFNetwork(num_students, num_topics, student_embedding_dim, topic_embedding_dim)

/home/david/anaconda3/envs/mlbd/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:197: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  rank_zero_warn(


In [29]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=ncf, train_dataloaders=train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                    | Type       | Params
-------------------------------------------------------
0 | student_embedding_layer | Embedding  | 5.3 K 
1 | topic_embedding_layer   | Embedding  | 1.6 K 
2 | network                 | Sequential | 2.8 K 
3 | loss                    | BCELoss    | 0     
-------------------------------------------------------
9.7 K     Trainable params
0         Non-trainable params
9.7 K     Total params
0.039     Total estimated model params size (MB)
/home/david/anaconda3/envs/mlbd/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoade

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
